In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import keras.utils.np_utils as kutils
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, MaxPool2D, Flatten, Dropout
from keras.callbacks import EarlyStopping



train_data_file = 'train.csv'
test_data_file = 'test.csv'

def open_csv():
    
    data1 = pd.read_csv(train_data_file )
    data2 = pd.read_csv(test_data_file)

    train_data = data1.values[0:, 1:]
    train_label = data1.values[0:, 0] 
    test_data = data2.values[0:, 0:]  
    return train_data, train_label, test_data

X_train, Y_train, Xtest = open_csv()

print(X_train.shape)
print(Y_train.shape)
print(Xtest.shape)

(42000, 784)
(42000,)
(28000, 784)


In [4]:
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
Xtest = Xtest.reshape(Xtest.shape[0], 28, 28,1)

print(X_train.shape)
print(Xtest.shape)

(42000, 28, 28, 1)
(28000, 28, 28, 1)


In [5]:
X_train = X_train / 255
Xtest = Xtest / 255
Y_train =  kutils.to_categorical(Y_train) 

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_train,Y_train,test_size = 0.3, random_state=2)

In [7]:
    model = Sequential()
    model.add(Conv2D(input_shape=(28, 28, 1), kernel_size=(3, 3), filters=32, padding='same', activation='relu'))
    model.add(Conv2D(kernel_size=(3, 3), filters=32, padding='same', activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
  
    model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor='accuracy', patience=2)
    model.fit(X_train, Y_train, epochs=4,  batch_size=128, verbose=1,callbacks=[early_stopping])



Epoch 1/4
29400/29400 [==============================] - 28s 951us/step - loss: 0.3829 - accuracy: 0.8817
Epoch 2/4
29400/29400 [==============================] - 26s 888us/step - loss: 0.1249 - accuracy: 0.9622
Epoch 3/4
29400/29400 [==============================] - 27s 927us/step - loss: 0.0878 - accuracy: 0.9733
Epoch 4/4
29400/29400 [==============================] - 27s 919us/step - loss: 0.0737 - accuracy: 0.9778


In [15]:
loss, accuracy = model.evaluate(X_test,Y_test, verbose=0)
print(loss,accuracy)

0.048901488803741004 0.9855555295944214


In [21]:
y_Pred = model.predict_classes(Xtest,verbose=0)

np.savetxt('tp_1_Digit_Recognizer.csv', np.c_[range(1,len(y_Pred)+1),y_Pred], delimiter=',', header = 'ImageId,Label', comments = '', fmt='%d')